In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import scipy
from PIL import Image 
from keras.datasets import mnist
from keras import layers
from keras.layers import Dropout
from matplotlib import pyplot as plt
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import initializers
from keras import optimizers
from matplotlib import pyplot
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.layers.normalization import BatchNormalization
from array import array
import cv2
import os
import random
from google.colab.patches import cv2_imshow

In [ ]:
new_shape = (28,28)
def load_images_from_folder(folder,images,y,i):
  
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        
        #img = Image.open(os.path.join(folder,filename))
        #print(type(img))
        if img is not None:
            #print(img.shape)
            #plt.imshow(img)
            img = cv2.resize(img,new_shape)
            images.append(img) 
            
            im=img.copy()
            mean=0
            st=0.7
            gauss = np.random.normal(mean,st,im.shape)
            gauss = gauss.astype('uint8')
            im = cv2.add(im,gauss)
            images.append(im)

            ima=img.copy()
            value = np.random.choice(np.array([-50, -40, -30, 30, 40, 50]))
            hsv = cv2.cvtColor(ima, cv2.COLOR_BGR2HSV)
            h, s, v = cv2.split(hsv)
            if value >= 0:
              lim = 255 - value
              v[v > lim] = 255
              v[v <= lim] += value
            else:
              lim = np.absolute(value)
              v[v < lim] = 0
              v[v >= lim] -= np.absolute(value)

            final_hsv = cv2.merge((h, s, v))
            ima = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
          
            im1=img.copy()
            value = np.random.choice(np.array([-50, -40, -30, 30, 40, 50]))
            hsv = cv2.cvtColor(im1, cv2.COLOR_BGR2HSV)
            h, s, v = cv2.split(hsv)
            if value >= 0:
              lim = 255 - value
              s[s > lim] = 255
              s[s <= lim] += value
            else:
              lim = np.absolute(value)
              s[s < lim] = 0
              s[s >= lim] -= np.absolute(value)

            final_hsv = cv2.merge((h, s, v))
            im1 = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

            im2=img.copy()
            brightness = 10
            contrast = random.randint(40, 100)
            dummy = np.int16(im2)
            dummy = dummy * (contrast/127+1) - contrast + brightness
            dummy = np.clip(dummy, 0, 255)
            im2 = np.uint8(dummy)
            images.append(im2)

            images.append(ima)
            images.append(im1)
            y.append(i-1)
            y.append(i-1)
            y.append(i-1)
            y.append(i-1)
            y.append(i-1)
            #i=i+1
    #print(i)
    return images,y

X = []
y=[]
for i in range(1,11):
  #print(i)
  if (i<10):
    folder="/content/gdrive/MyDrive/trainPart1/train/Sample00"+str(i)
  else:
    folder="/content/gdrive/MyDrive/trainPart1/train/Sample0"+str(i)
  X,y=load_images_from_folder(folder,X,y,i)
  


In [ ]:
X=np.array(X)
print(X.shape)
y=np.array(y)
print(y.shape)
train_images, test_imagesall, train_labels, test_labelsall = train_test_split(X, y, test_size=0.15, random_state=42)
print('Shape of images is..',train_images.shape)
print('Shape of labels is..',train_labels.shape)


val_images = test_imagesall[0:200,:,:]
val_labels = test_labelsall[0:200]
test_images = test_imagesall[200:,:,:]
test_labels = test_labelsall[200:]


# display sample
print("Shape of training images:",train_images.shape)
print("Shape of training labels:",train_labels.shape)
print("unique labels:",np.unique(train_labels))
print("Shape of val images:",val_images.shape)
print("Shape of val labels:",val_labels.shape)
print("Shape of test images:",test_images.shape)
print("Shape of test labels:",test_labels.shape)

(2000, 28, 28, 3)
(2000,)
Shape of images is.. (1700, 28, 28, 3)
Shape of labels is.. (1700,)
Shape of training images: (1700, 28, 28, 3)
Shape of training labels: (1700,)
unique labels: [0 1 2 3 4 5 6 7 8 9]
Shape of val images: (200, 28, 28, 3)
Shape of val labels: (200,)
Shape of test images: (100, 28, 28, 3)
Shape of test labels: (100,)


In [ ]:
train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5
val_images = (val_images / 255) - 0.5
print(train_images.shape)

(1700, 28, 28, 3)


In [ ]:
model = Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(28,28,3)))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))

model.add(layers.Dense(units=84, activation='relu'))

model.add(layers.Dense(units=10, activation = 'softmax'))

In [ ]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

model.compile(
  optimizer=adam,
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# obtain the summary
model.summary()


es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 5)
mc = ModelCheckpoint('/content/gdrive/MyDrive/models/pretrained_2.h5', monitor='val_accuracy', mode='max', 
                     verbose=1, save_best_only=True)
print(train_images.shape)
print(val_images.shape)
# Train the model.


train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )

val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)

test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(train_images)
val_generator.fit(val_images)
test_generator.fit(test_images)

#AlexNet.fit_generator(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs = epochs, steps_per_epoch = x_train.shape[0]//batch_size, validation_data = val_generator.flow(x_val, y_val, batch_size=batch_size), validation_steps = 250, callbacks = [lrr], verbose=1)

history=model.fit(train_generator.flow(
  train_images,
  to_categorical(train_labels),batch_size=32),
  validation_data=val_generator.flow(val_images, to_categorical(val_labels),batch_size=32),  
  epochs=20,
  shuffle = True,
  callbacks=[es,mc],
)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 6)         168       
_________________________________________________________________
average_pooling2d (AveragePo (None, 13, 13, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 16)        880       
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               48120     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [ ]:
scores = model.evaluate(
  test_images,
  to_categorical(test_labels)
)

print('Test accuracy:', scores[1])

4/4 [==============================] - 0s 5ms/step - loss: 0.1265 - accuracy: 0.9600
Test accuracy: 0.9599999785423279
